# song crawler

In [21]:
import requests
from bs4 import BeautifulSoup
pages = range(1,1730)
menu_page = 'https://lyricvn.com/tag/viet-nam/page/{}/'
song_pages = set()
for page_idx in pages:
    url = menu_page.format(page_idx)
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, "html.parser")
    for tag in soup.find_all('a'):
        if 'lyricvn.com/loi-bai-hat' in tag.get('href'):
            song_pages.add(tag.get('href'))
    if page_idx % 100 == 0:
        print('Processed {} pages'.format(page_idx))

Processed 100 pages
Processed 200 pages
Processed 300 pages
Processed 400 pages
Processed 500 pages
Processed 600 pages
Processed 700 pages
Processed 800 pages
Processed 900 pages
Processed 1000 pages
Processed 1100 pages
Processed 1200 pages
Processed 1300 pages
Processed 1400 pages
Processed 1500 pages
Processed 1600 pages
Processed 1700 pages


In [73]:
import pandas as pd
list_song_pages = list(song_pages)
file_paths = []
segment_file_paths = []
chunk = 0 
for url_idx,url in enumerate(list_song_pages):
    if url_idx % 100 == 0:
        chunk += 1
    file_name = url.replace('https://lyricvn.com/','').replace('/','')+'.txt'
    file_paths.append(f"raw/chunk_{chunk}/{file_name}")
    segment_file_paths.append(f"segment/chunk_{chunk}/segment_{file_name}")
    

pd_songs = pd.DataFrame(list_song_pages,columns=['song_url'])
pd_songs['file_path'] = file_paths
pd_songs['segment_file_path'] = segment_file_paths
pd_songs.to_csv('data/song_from_lyricvn_com/songs_meta_data.csv',index=False)

In [65]:
import pandas as pd
import os
loaded_pd_songs = pd.read_csv('data/songs_meta_data.csv')
for idx, (url, file_path) in enumerate(loaded_pd_songs.values):
    abs_file_path = "data/"+file_path
    if os.path.exists(abs_file_path):
        continue
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, "html.parser")
    os.makedirs(os.path.dirname(abs_file_path), exist_ok=True)
    with open(abs_file_path,'w+') as f:
        for tag in soup.find_all('p'):
            br_tags = tag.find_all('br')
            for br_tag in br_tags:
                br_tag.replace_with("\n")
            content = tag.get_text() +'\n'
            if len(content) > 2:
                f.write(content+'\n')
    if idx %100 == 0:
        print('Processed {} pages'.format(idx))
    

Processed 100 pages
Processed 200 pages
Processed 300 pages
Processed 400 pages
Processed 500 pages
Processed 600 pages
Processed 700 pages
Processed 800 pages
Processed 900 pages
Processed 1000 pages
Processed 1100 pages
Processed 1200 pages
Processed 1300 pages
Processed 1400 pages
Processed 1500 pages
Processed 1600 pages
Processed 1700 pages
Processed 1800 pages
Processed 1900 pages
Processed 2000 pages
Processed 2100 pages
Processed 2200 pages
Processed 2300 pages
Processed 2400 pages
Processed 2500 pages
Processed 2600 pages
Processed 2700 pages
Processed 2800 pages
Processed 2900 pages
Processed 3000 pages
Processed 3100 pages
Processed 3200 pages
Processed 3300 pages
Processed 3400 pages
Processed 3500 pages
Processed 3600 pages
Processed 3700 pages
Processed 3800 pages
Processed 3900 pages
Processed 4000 pages
Processed 4100 pages
Processed 4200 pages
Processed 4300 pages
Processed 4400 pages
Processed 4500 pages
Processed 4600 pages
Processed 4700 pages
Processed 4800 pages
P

# novel cralwer

In [2]:

import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

link_truyen:str = 'https://truyen.tangthuvien.vn/doc-truyen/{ten_truyen}/chuong-{chuong}'
so_chuong_truyen = {
    "ta-xac-chet-phong-dang-khong-bi-troi-buoc" : 151,
    "thien-dao-do-thu-quan-convert": 2264,
    "tu-chan-lieu-thien-quan": 3171,
    "xuyen-nhanh-nam-than-co-chu%CC%81t-chay!" : 2652,
    "nu-phu-khong-lan-vao-khoai-xuyen" : 475,
    "dap-noi-ban-sat-di-hoc" : 337,
    "xuyen-thu-tho-he-kho-nu" : 252,
    "nhan-vat-phan-dien-hom-nay-cung-that-ngoan" : 1192,
    "khoai-xuyen-he-thong-nhan-vat-phan-dien-boss-dot-kich!" : 1666,
    "tieu-yeu-the" : 758,
    "su-phu-lai-rot-tuyen" : 704,
    "khoai-xuyen-moi-lan-deu-la-ta-nam-thuong" : 2020,
    "khai-cuc-nu-ma-dau-phu-ta": 1144,
    "than-dao-dan-ton": 4934,
    "cau-tai-yeu-vu-loan-the-tu-tien": 959,
    "so-menh-chi-hoan-tuc-menh-chi-hoan": 207,
    "thuy-nhuong-tha-tu-tien-dich!--ai-bao-han-tu-tien!": 458,
    "moi-ngay-deu-cach-hien-hinh-cang-gan-mot-buoc-moi-thien-do-ly-hien-hinh-canh-can-nhat-bo": 503,
    "cac-nguoi-tu-tien-ta-lam-ruong": 444,
    "ta-co-mot-vien-xuc-xac-van-menh-ma-thuat": 276,
    "di-hoa-do-thi": 2500,
    "my-thuc-cung-ung-thuong": 2734,
    "di-thuong-sinh-vat-kien-van-luc": 1778,
}

base_data = '/home/giangtran/working/bavandangvanba/data/truyen_from_tangthuvien_vn/'
cols = ['ten_truyen','truyen_url','file_path','segment_file_path']
truyen_meta_data = []
for ten_truyen in so_chuong_truyen:
    abs_folder = base_data+ten_truyen+'/'
    os.makedirs(os.path.dirname(abs_folder), exist_ok=True)
    os.makedirs(os.path.dirname(abs_folder+'raw/'), exist_ok=True)
    os.makedirs(os.path.dirname(abs_folder+'segment/'), exist_ok=True)
    for chuong in range(1,so_chuong_truyen[ten_truyen] + 1):
        raw_file_path  = abs_folder+f"raw/chuong_{chuong}"+'.txt'
        segment_file_paths  = abs_folder+f"segment/chuong_{chuong}"+'.txt'
        link = link_truyen.format(ten_truyen=ten_truyen,chuong=chuong)
        truyen_meta_data.append([ten_truyen,link,raw_file_path,segment_file_paths])

        if os.path.exists(raw_file_path) :
            print(f"Skip {chuong} chuong : {ten_truyen}")
            continue
        response = requests.get(link)
        content = response.content
        soup = BeautifulSoup(content, "html.parser")
        
        tags = soup.find_all('div', class_=lambda x: x and x.startswith('box-chap'))
        text= ""
        for tag in tags:
            text += tag.get_text() + " "
        with open(raw_file_path,'w+') as f:
            f.write(text)
    print(f"Done {chuong} chuong : {ten_truyen}")

pd_truyen = pd.DataFrame(truyen_meta_data,columns=cols)
pd_truyen.to_csv('/home/giangtran/working/bavandangvanba/data/truyen_from_tangthuvien_vn/truyen_meta_data.csv',index=False)

Skip 1 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 2 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 3 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 4 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 5 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 6 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 7 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 8 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 9 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 10 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 11 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 12 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 13 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 14 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 15 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 16 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 17 chuong : ta-xac-chet-phong-dang-khong-bi-troi-buoc
Skip 1

In [12]:
for i in so_chuong_truyen:
    print(i)

ta-xac-chet-phong-dang-khong-bi-troi-buoc
thien-dao-do-thu-quan-convert
tu-chan-lieu-thien-quan
xuyen-nhanh-nam-than-co-chu%CC%81t-chay!
nu-phu-khong-lan-vao-khoai-xuyen
dap-noi-ban-sat-di-hoc
xuyen-thu-tho-he-kho-nu
nhan-vat-phan-dien-hom-nay-cung-that-ngoan
khoai-xuyen-he-thong-nhan-vat-phan-dien-boss-dot-kich!
tieu-yeu-the
su-phu-lai-rot-tuyen
khoai-xuyen-moi-lan-deu-la-ta-nam-thuong


In [69]:
loaded_pd_songs = pd.read_csv('data/song_from_lyricvn_com/songs_meta_data.csv')
for idx, (url, file_path) in enumerate(loaded_pd_songs.values):
    abs_file_path = "data/"+file_path
    content = open(abs_file_path,'r').read()
    break

In [71]:
print(content)

Tên bài hát: Cứ Ngủ Say
 Ca sĩ: Khởi My, Chế Đình Cường
 Sáng tác: unknown
 Album: Sao Đom Đóm
 Ngày ra mắt: 01/01/1970
 Thể loại: Việt Nam, Nhạc Trẻ, V-Pop

Bài Hát: Cứ́ Ngủ Say
 Ca Sĩ: Khởi My

Lắng nghe
 từng giọt nắng lên đầy
 Lắng nghe
 mùa thu đến sớm nay
 Có lẽ lúc này
 anh yêu vẫn ngủ say
 Lòng em nhớ anh
 thật nhiều

Lắng nghe
 từng góc phố ồn ào
 Hát lên
 lòng vui sướng biết bao
 Đến lúc thức dậy
 anh yêu em biết mấy
 Lòng anh nhớ em
 thật nhiều

Dẫu biết có lúc giận hờn
 có lúc ngại ngần
 Vẫn thấy tim em
 gần anh hơn
 Nhớ lắm ánh mắt còn gầy
 tiếng hát còn đầy
 Vẫn thấy trong em
 cơn ngủ say
 Có lắm ước muốn tràn trề
 phút chốc ùa về
 phút chốc qua như
 là cơn mê
 Vẫn biết có lúc nhìn lại
 những tháng ngày dài
 Ánh mắt ban mai
 dành tặng anh

Lắng nghe
 từng giọt nắng lên đầy
 Lắng nghe
 mùa thu đến sớm nay
 Có lẽ lúc này
 anh yêu vẫn ngủ say
 Lòng em nhớ anh
 thật nhiều

Lắng nghe
 từng góc phố ồn ào
 Hát lên
 lòng vui sướng biết bao
 Đến lúc thức dậy
 anh yêu em biết mấy
 

In [1]:
so_chuong_truyen = {
    "ta-xac-chet-phong-dang-khong-bi-troi-buoc" : 151,
    "thien-dao-do-thu-quan-convert": 2264,
    "tu-chan-lieu-thien-quan": 3171,
    "xuyen-nhanh-nam-than-co-chu%CC%81t-chay!" : 2652,
    "nu-phu-khong-lan-vao-khoai-xuyen" : 475,
    "dap-noi-ban-sat-di-hoc" : 337,
    "xuyen-thu-tho-he-kho-nu" : 252,
    "nhan-vat-phan-dien-hom-nay-cung-that-ngoan" : 1192,
    "khoai-xuyen-he-thong-nhan-vat-phan-dien-boss-dot-kich!" : 1666,
    "tieu-yeu-the" : 758,
    "su-phu-lai-rot-tuyen" : 704,
    "khoai-xuyen-moi-lan-deu-la-ta-nam-thuong" : 2020,
    "khai-cuc-nu-ma-dau-phu-ta": 1144,
    "cau-tai-yeu-vu-loan-the-tu-tien": 932,
    "than-dao-dan-ton": 4934,
}
sum(map(lambda x: so_chuong_truyen[x], so_chuong_truyen))

22652